<a href="https://colab.research.google.com/github/nisbenz/Mini-Transformer/blob/main/Encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving simplemath_100k_balanced.csv to simplemath_100k_balanced.csv


In [ ]:
import pandas as pd
import io

file_name = 'simplemath_100k_balanced.csv'
ds = pd.read_csv(io.BytesIO(uploaded[file_name]))

print(f"Successfully loaded '{file_name}' into a DataFrame.")
display(ds.head())

Successfully loaded 'simplemath_100k_balanced.csv' into a DataFrame.


,problem,answer
0,2790 + 6698 =,9488
1,8262 + 3848 =,12110
2,1628 + 132 =,1760
3,9925 + 7169 =,17094
4,3023 + 6929 =,9952


In [ ]:
import pandas as pd
X_train = pd.Series(ds["problem"])
X_train = X_train.str.ljust(13, ' ')

raw_chars = sorted(list(set(''.join(X_train))))

vocab =  raw_chars

stoi = { ch:i for i,ch in enumerate(vocab) }
itos = { i:ch for i,ch in enumerate(vocab) }

print(f"Vocabulary Size: {len(vocab)}")
print("Character Map:", stoi)
def encode_problem(problem_str):
    return [stoi[c] for c in problem_str]

X_train_encoded = [encode_problem(x) for x in X_train]

print("\nExample Output:")
print(f"Original: '{X_train[0]}'")
print(f"Encoded:  {X_train_encoded[0]}")




Vocabulary Size: 19
Character Map: {'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, ' ': 3, '+': 4, '-': 5, '/': 6, '0': 7, '1': 8, '2': 9, '3': 10, '4': 11, '5': 12, '6': 13, '7': 14, '8': 15, '9': 16, '=': 17, 'x': 18}

Example Output:
Original: '2790 + 6698 ='
Encoded:  [9, 14, 16, 7, 3, 4, 3, 13, 13, 16, 15, 3, 17]
